In [1]:
import numpy
import vectorbt as vbt
import warnings
from numba import njit

import talib
warnings.filterwarnings("ignore")

In [2]:
symbols = ["BTC-USD","ETH-USD"]
fast_window, slow_window, rsi_window, top, bottom = (180,240, 14, 80,20)
fees = 0.001
interval = "1D"
start = "2020-01-01"

In [3]:
data = vbt.YFData.download(symbols,
                           start=start, 
                           interval=interval).concat()
data["Close"]

symbol,BTC-USD,ETH-USD
Date,,
2019-12-31 00:00:00+00:00,7193.599121,129.610855
2020-01-01 00:00:00+00:00,7200.174316,130.802002
2020-01-02 00:00:00+00:00,6985.470215,127.410179
2020-01-03 00:00:00+00:00,7344.884277,134.171707
2020-01-04 00:00:00+00:00,7410.656738,135.069366
...,...,...
2023-08-22 00:00:00+00:00,26031.656250,1633.892578
2023-08-23 00:00:00+00:00,26431.640625,1679.274414
2023-08-24 00:00:00+00:00,26162.373047,1659.944580


In [4]:
data

{'Open': symbol                          BTC-USD      ETH-USD
 Date                                                
 2019-12-31 00:00:00+00:00   7294.438965   132.612274
 2020-01-01 00:00:00+00:00   7194.892090   129.630661
 2020-01-02 00:00:00+00:00   7202.551270   130.820038
 2020-01-03 00:00:00+00:00   6984.428711   127.411263
 2020-01-04 00:00:00+00:00   7345.375488   134.168518
 ...                                 ...          ...
 2023-08-22 00:00:00+00:00  26130.748047  1667.279297
 2023-08-23 00:00:00+00:00  26040.474609  1634.401733
 2023-08-24 00:00:00+00:00  26431.519531  1679.248291
 2023-08-25 00:00:00+00:00  26163.679688  1660.269897
 2023-08-26 00:00:00+00:00  26046.810547  1652.837769
 
 [1335 rows x 2 columns],
 'High': symbol                          BTC-USD      ETH-USD
 Date                                                
 2019-12-31 00:00:00+00:00   7335.290039   133.732681
 2020-01-01 00:00:00+00:00   7254.330566   132.835358
 2020-01-02 00:00:00+00:00   7212.1552

In [5]:
# create custom bbands indicator
def bbands_indicator(close, timeperiod, nbdevup, nbdevdn, matype):
    # bbands strategy
    middle = numpy.full(close.shape, numpy.nan, dtype=numpy.float_)
    upper = numpy.full(close.shape, numpy.nan, dtype=numpy.float_)
    lower = numpy.full(close.shape, numpy.nan, dtype=numpy.float_)
    for col in range(close.shape[1]):
        upper[:,col], middle[:,col], lower[:, col] =\
            talib.BBANDS(close[:,col], timeperiod=timeperiod, 
                         nbdevup=nbdevup, nbdevdn=nbdevdn, 
                         matype=matype)
    return upper, middle, lower

BBANDS_factory = vbt.IndicatorFactory(
    input_names=['close'],
    param_names=['timeperiod','nbdevup','nbdevdn','matype'],
    output_names=['upper', "middle", "lower"]
).from_apply_func(bbands_indicator,
                  timeperiod=14,
                  nbdevup=2,
                  nbdevdn=2,
                  matype=0)

bbands = BBANDS_factory.run(
    data["Close"],
)
bbands.upper


symbol,BTC-USD,ETH-USD
Date,,
2019-12-31 00:00:00+00:00,NaN,NaN
2020-01-01 00:00:00+00:00,NaN,NaN
2020-01-02 00:00:00+00:00,NaN,NaN
2020-01-03 00:00:00+00:00,NaN,NaN
2020-01-04 00:00:00+00:00,NaN,NaN
...,...,...
2023-08-22 00:00:00+00:00,31073.548124,1939.559530
2023-08-23 00:00:00+00:00,30808.749690,1924.908505
2023-08-24 00:00:00+00:00,30523.106406,1908.380232


In [6]:
# create custom bbands indicator
def bbands_indicator_vbt(Open, High, Low, Close, timeperiod, 
                         nbdevup, nbdevdn, matype):
    bbands = vbt.talib("BBANDS").run(Close, timeperiod=timeperiod,
                                     nbdevup=nbdevup, nbdevdn=nbdevdn,
                                     matype=matype)
    upper, middle, lower = bbands.upperband,\
                            bbands.middleband,\
                            bbands.lowerband
    return upper, middle, lower

BBANDS_factory_vbt = vbt.IndicatorFactory(
    input_names=['Open', 'High', 'Low', 'Close'],
    param_names=['timeperiod','nbdevup','nbdevdn', 'matype'],
    output_names=['upper', "middle", "lower"]
).from_apply_func(bbands_indicator_vbt,
                  timeperiod=14,
                  nbdevup=2,
                  nbdevdn=2,
                  matype=0)

bbands_vbt = BBANDS_factory_vbt.run(
    data["Open"],
    data["High"],
    data["Low"],
    data["Close"],
    
)
bbands_vbt.upper


symbol,BTC-USD,ETH-USD
Date,,
2019-12-31 00:00:00+00:00,NaN,NaN
2020-01-01 00:00:00+00:00,NaN,NaN
2020-01-02 00:00:00+00:00,NaN,NaN
2020-01-03 00:00:00+00:00,NaN,NaN
2020-01-04 00:00:00+00:00,NaN,NaN
...,...,...
2023-08-22 00:00:00+00:00,31073.548124,1939.559530
2023-08-23 00:00:00+00:00,30808.749690,1924.908505
2023-08-24 00:00:00+00:00,30523.106406,1908.380232


In [7]:
data["Close"]

symbol,BTC-USD,ETH-USD
Date,,
2019-12-31 00:00:00+00:00,7193.599121,129.610855
2020-01-01 00:00:00+00:00,7200.174316,130.802002
2020-01-02 00:00:00+00:00,6985.470215,127.410179
2020-01-03 00:00:00+00:00,7344.884277,134.171707
2020-01-04 00:00:00+00:00,7410.656738,135.069366
...,...,...
2023-08-22 00:00:00+00:00,26031.656250,1633.892578
2023-08-23 00:00:00+00:00,26431.640625,1679.274414
2023-08-24 00:00:00+00:00,26162.373047,1659.944580


In [8]:
import pandas
from datetime import datetime
mask = pandas.DataFrame([
    [True, False, False],
    [False, True, False],
    [False, False, True],
    [True, False, False],
    [False, True, False]
], index=pandas.Index([
    datetime(2020, 1, 1),
    datetime(2020, 1, 2),
    datetime(2020, 1, 3),
    datetime(2020, 1, 4),
    datetime(2020, 1, 5)
]), columns=['a', 'b', 'c'])

ts = pandas.Series([3., 4., 3.5, 3., 1.], index=mask.index)
ts

2020-01-01    3.0
2020-01-02    4.0
2020-01-03    3.5
2020-01-04    3.0
2020-01-05    1.0
dtype: float64

In [20]:
# enter after current 
out_dict={}
entries, exits = mask.vbt.signals.generate_ohlc_stop_exits(ts, sl_stop=0.04,sl_trail=False,
                                                           tp_stop=0.1, out_dict=out_dict,
                                                            chain=True) # remove ignored entry signals
entries


,a,b,c
2020-01-01,True,False,False
2020-01-02,False,True,False
2020-01-03,False,False,True
2020-01-04,True,False,False
2020-01-05,False,True,False


In [21]:
exits

,a,b,c
2020-01-01,False,False,False
2020-01-02,True,False,False
2020-01-03,False,True,False
2020-01-04,False,False,True
2020-01-05,True,False,False


In [10]:
# Reference: https://github.com/polakowo/vectorbt/issues/64
# buy limit, sell limit 
import vectorbt as vbt

# limit = -0.01 means 1% stop
fast, slow = vbt.MA.run_combs(data["Close"], window=[12,24])

entries = fast.ma_crossed_above(slow)
# add buy limit order below 3% of entry price
entries_with_limit_order = entries.vbt.signals.generate_ohlc_stop_exits(
                            data["Close"], sl_stop=0.03, sl_trail=False
                            )
exits = fast.ma_crossed_below(slow)

pf2 = vbt.Portfolio.from_signals(data["Close"],
                           entries=entries_with_limit_order,
                           exits=exits,
                           open=data["Open"],
                           high=data["High"],
                           low=data["Low"],
                           sl_stop=0.07,
                           sl_trail=False,
                           tp_stop=0.14,
                           stop_entry_price="price")

pf2.stats()

Start                         2019-12-31 00:00:00+00:00
End                           2023-08-26 00:00:00+00:00
Period                               1335 days 00:00:00
Start Value                                       100.0
End Value                                    119.764483
Total Return [%]                              19.764483
Benchmark Return [%]                         716.863602
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              31.319753
Max Drawdown Duration                 603 days 12:00:00
Total Trades                                       16.5
Total Closed Trades                                16.5
Total Open Trades                                   0.0
Open Trade PnL                                      0.0
Win Rate [%]                                  45.772059
Best Trade [%]                                     14.0
Worst Trade [%]                                 

In [11]:
# Reference: https://github.com/polakowo/vectorbt/issues/64
# buy limit, sell limit 

# limit = -0.01 means 1% stop
fast, slow = vbt.MA.run_combs(data["Close"], window=[12,24])

entries = fast.ma_crossed_above(slow)
exits = fast.ma_crossed_below(slow)

pf3 = vbt.Portfolio.from_signals(data["Close"],
                           entries=entries,
                           exits=exits,
                           open=data["Open"],
                           high=data["High"],
                           low=data["Low"],
                           direction="LongOnly",
                           sl_stop=0.07, # A percentage below/above the acquisition price for long/short position.
                           tp_stop=0.14, # A percentage above/below the acquisition price for long/short position. Note that 0.01 = 1%.
                           sl_trail=False,
                           stop_entry_price="fillprice", ## StopEntryPrice https://vectorbt.dev/api/portfolio/enums/#vectorbt.portfolio.enums.StopEntryPrice
                           stop_exit_price="StopLimit",
                           ) ## https://vectorbt.dev/api/portfolio/enums/#vectorbt.portfolio.enums.StopExitPrice

pf3.stats()

Start                         2019-12-31 00:00:00+00:00
End                           2023-08-26 00:00:00+00:00
Period                               1335 days 00:00:00
Start Value                                       100.0
End Value                                    151.946256
Total Return [%]                              51.946256
Benchmark Return [%]                         716.863602
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              34.881595
Max Drawdown Duration                 595 days 12:00:00
Total Trades                                       30.0
Total Closed Trades                                30.0
Total Open Trades                                   0.0
Open Trade PnL                                      0.0
Win Rate [%]                                  44.117647
Best Trade [%]                                     14.0
Worst Trade [%]                                 

In [12]:
pf3.trades.records_readable

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,"(12, 24, BTC-USD)",0.015531,2020-03-31 00:00:00+00:00,6438.644531,0.0,2020-04-07 00:00:00+00:00,7340.054766,0.0,14.000000,0.140000,Long,Closed,0
1,1,"(12, 24, BTC-USD)",0.011962,2020-06-02 00:00:00+00:00,9529.803711,0.0,2020-06-19 00:00:00+00:00,9288.018555,0.0,-2.892348,-0.025371,Long,Closed,1
2,2,"(12, 24, BTC-USD)",0.012086,2020-07-15 00:00:00+00:00,9192.836914,0.0,2020-07-27 00:00:00+00:00,10479.834082,0.0,15.555071,0.140000,Long,Closed,2
3,3,"(12, 24, BTC-USD)",0.012362,2020-09-23 00:00:00+00:00,10246.186523,0.0,2020-10-02 00:00:00+00:00,10575.974609,0.0,4.076820,0.032186,Long,Closed,3
4,4,"(12, 24, BTC-USD)",0.012113,2020-10-05 00:00:00+00:00,10793.339844,0.0,2020-10-06 00:00:00+00:00,10604.406250,0.0,-2.288549,-0.017505,Long,Closed,4
5,5,"(12, 24, BTC-USD)",0.011371,2020-10-10 00:00:00+00:00,11296.361328,0.0,2020-10-21 00:00:00+00:00,12877.851914,0.0,17.983139,0.140000,Long,Closed,5
6,6,"(12, 24, BTC-USD)",0.003729,2021-02-06 00:00:00+00:00,39266.011719,0.0,2021-02-08 00:00:00+00:00,44763.253359,0.0,20.500779,0.140000,Long,Closed,6
7,7,"(12, 24, BTC-USD)",0.002912,2021-03-12 00:00:00+00:00,57332.089844,0.0,2021-03-24 00:00:00+00:00,53318.843555,0.0,-11.685444,-0.070000,Long,Closed,7
8,8,"(12, 24, BTC-USD)",0.002629,2021-04-05 00:00:00+00:00,59057.878906,0.0,2021-04-18 00:00:00+00:00,54923.827383,0.0,-10.867463,-0.070000,Long,Closed,8
9,9,"(12, 24, BTC-USD)",0.002455,2021-05-08 00:00:00+00:00,58803.777344,0.0,2021-05-10 00:00:00+00:00,54687.512930,0.0,-10.106740,-0.070000,Long,Closed,9


In [13]:
pf3.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(12, 24, BTC-USD)",2020-03-31 00:00:00+00:00,0.015531,6438.644531,0.0,Buy
1,1,"(12, 24, BTC-USD)",2020-04-07 00:00:00+00:00,0.015531,7340.054766,0.0,Sell
2,2,"(12, 24, BTC-USD)",2020-06-02 00:00:00+00:00,0.011962,9529.803711,0.0,Buy
3,3,"(12, 24, BTC-USD)",2020-06-19 00:00:00+00:00,0.011962,9288.018555,0.0,Sell
4,4,"(12, 24, BTC-USD)",2020-07-15 00:00:00+00:00,0.012086,9192.836914,0.0,Buy
...,...,...,...,...,...,...,...
115,115,"(12, 24, ETH-USD)",2023-06-10 00:00:00+00:00,0.113536,1760.562656,0.0,Sell
116,116,"(12, 24, ETH-USD)",2023-06-26 00:00:00+00:00,0.107499,1859.432861,0.0,Buy
117,117,"(12, 24, ETH-USD)",2023-07-17 00:00:00+00:00,0.107499,1911.646118,0.0,Sell
118,118,"(12, 24, ETH-USD)",2023-07-18 00:00:00+00:00,0.108295,1897.599243,0.0,Buy
